# Expo Analyse

### Bibs

In [35]:
import pandas as pd
import numpy as np

### Daten Import

In [36]:
art_hi = pd.read_pickle('../data/artikelhierarchie.pkl')
item_clu = pd.read_pickle('../data/item_cluster.pkl')
visits = pd.read_pickle('../data/visit_h.pkl')

In [37]:
art_hi.head(2)

,BDF,MERCHANDISESTYLE_CODE,MERCHANDISESTYLE_COMMUNICATIONKEY,PRODUCT_ID,MARKE,MARKENGRUPPE,LAGERWARENTYP,VERWENDUNGSKENNZEICHEN,PRODUKTTYP,PBK,...,SUB_KATEGORIE_ID,SUB_KATEGORIE_BEZEICHNUNG,KATEGORIE_ID,KATEGORIE_BEZEICHNUNG,KATEGORIE_DIRECT_ID,KATEGORIE_DIRECT_BEZEICHNUNG,NOS,VEK,BEK,STAMM_VK
0,0,JANAGELENK,1233553071,1233553071,JANASTYLE,Fremdmarke,LW,1,Armstulpen,Stulpen,...,101,Mainstream,11,Fashion,1,Fashion & Sports,Nein,5.9,5.9,14.99
1,0,JANAGELENK,1233553071,1233553071,JANASTYLE,Fremdmarke,LW,1,Armstulpen,Stulpen,...,101,Mainstream,11,Fashion,1,Fashion & Sports,Nein,5.9,5.9,14.99


In [38]:
item_clu.head(2)

,SUB_KATEGORIE_ID,ITEM_H_SK,GESCHAEFTSJAHR_HALBJAHR,MONAT,IS_DEALBUY,DEALBUY_VALUE,IS_NOS,NOS_VALUE,SEASON,SEASONORIGIN,WITH_ORDER,CLUSTER_DIM1,CLUSTER_DIM2,RK
0,516,bcf3b916c89dbc4db300adebb3233588,202302,1,False,None,False,None,149,143,False,KOLLEKTION,BESTANDSUEBERNAHME/RESTE,1
1,516,47c13bbc6427ffc1dab98bd2e13cbd72,202402,1,False,None,False,None,149,140,False,KOLLEKTION,BESTANDSUEBERNAHME/RESTE,1


In [39]:
visits.head(2)

,visit_type,visit_h_sk,ot_bot
0,qVisit,b'\x11$\x11|\x84}}\x16\xee\x10\x01G\x02e/\x8f',False
1,qVisit,b'^?(\x85\\b\xfe\x80=(;Q\x9b\x04n0',False


### Explore 1
Tabellen kennen lernen für Bearbeitung 

shape all


In [40]:
print(f'art_hi: {art_hi.shape} \nitem_clu: {item_clu.shape} \nvisits: {visits.shape}')

art_hi: (1000, 40) 
item_clu: (1000, 14) 
visits: (6080250, 3)


 `art_hi` Daten kennen lernen für Bearbeitung

In [41]:
art_hi.columns

Index(['BDF', 'MERCHANDISESTYLE_CODE', 'MERCHANDISESTYLE_COMMUNICATIONKEY',
       'PRODUCT_ID', 'MARKE', 'MARKENGRUPPE', 'LAGERWARENTYP',
       'VERWENDUNGSKENNZEICHEN', 'PRODUKTTYP', 'PBK',
       'ITEMOPTION_COMMUNICATIONKEY', 'ITEMOPTION_H_SK', 'ARTIKELGROESSE',
       'VARIATION_ID', 'ARTIKELNUMMER', 'ITEM_COMMUNICATIONKEY', 'ITEM_H_SK',
       'URSPRUNG_SEASON', 'SEASON', 'LKZ', 'LIEFERANT_BEZEICHNUNG',
       'MARKENKENNZEICHEN', 'MKZ', 'MKZ_BEZEICHNUNG', 'WARENGRUPPE', 'FT',
       'OT', 'MARKTVERGLEICHSGRUPPE', 'ARTIKELSTATUS', 'SAISONKENNZEICHEN',
       'SUB_KATEGORIE_ID', 'SUB_KATEGORIE_BEZEICHNUNG', 'KATEGORIE_ID',
       'KATEGORIE_BEZEICHNUNG', 'KATEGORIE_DIRECT_ID',
       'KATEGORIE_DIRECT_BEZEICHNUNG', 'NOS', 'VEK', 'BEK', 'STAMM_VK'],
      dtype='object')

In [42]:
art_hi.isnull().sum()

BDF                                    0
MERCHANDISESTYLE_CODE                  0
MERCHANDISESTYLE_COMMUNICATIONKEY      0
PRODUCT_ID                           998
MARKE                                  0
MARKENGRUPPE                           0
LAGERWARENTYP                          0
VERWENDUNGSKENNZEICHEN                 0
PRODUKTTYP                             0
PBK                                    0
ITEMOPTION_COMMUNICATIONKEY            0
ITEMOPTION_H_SK                        0
ARTIKELGROESSE                         0
VARIATION_ID                         998
ARTIKELNUMMER                          0
ITEM_COMMUNICATIONKEY                  0
ITEM_H_SK                              0
URSPRUNG_SEASON                        0
SEASON                                 0
LKZ                                    0
LIEFERANT_BEZEICHNUNG                  0
MARKENKENNZEICHEN                      0
MKZ                                    0
MKZ_BEZEICHNUNG                        0
WARENGRUPPE     

 `item_clu` Daten kennen lernen für Bearbeitung

In [43]:
item_clu.isnull().sum()

SUB_KATEGORIE_ID             0
ITEM_H_SK                    0
GESCHAEFTSJAHR_HALBJAHR      0
MONAT                        0
IS_DEALBUY                   0
DEALBUY_VALUE              997
IS_NOS                       0
NOS_VALUE                  947
SEASON                       0
SEASONORIGIN                 1
WITH_ORDER                   0
CLUSTER_DIM1                 0
CLUSTER_DIM2                 0
RK                           0
dtype: int64

In [44]:
item_clu.columns

Index(['SUB_KATEGORIE_ID', 'ITEM_H_SK', 'GESCHAEFTSJAHR_HALBJAHR', 'MONAT',
       'IS_DEALBUY', 'DEALBUY_VALUE', 'IS_NOS', 'NOS_VALUE', 'SEASON',
       'SEASONORIGIN', 'WITH_ORDER', 'CLUSTER_DIM1', 'CLUSTER_DIM2', 'RK'],
      dtype='object')

`visits` Daten kennen lernen für Bearbeitung 

In [45]:
visits.columns

Index(['visit_type', 'visit_h_sk', 'ot_bot'], dtype='object')

In [46]:
visits.isnull().sum()

visit_type    0
visit_h_sk    0
ot_bot        0
dtype: int64

### Transform Data 

Tabellen Joinen
